# AI YOUTH CAMP

### Hendrik Santoso Sugiarto, Ph.D.
### Calvin Institute of Technology

Stable Diffusion
---

Stable diffusion merupakan model AI generatif yang menghasilkan gambar fotorealistik unik dari perintah teks dan gambar. Model ini mempelajari proses dalam menambah noise pada gambar realistik lalu kemudian mempelajari proses kebalikannya. Model menggunakan CLIP ViT untuk mempelajari representasi laten yang merelasikan gambar dan kata.


### Referensi

Data latih: database kumpulan gambar 512x512 [LAION-5B](https://laion.ai/blog/laion-5b/).

Model: CLIP ViT-L/14 (860M UNet + 123M text encoder) [model card](https://huggingface.co/CompVis/stable-diffusion).


### Install Library

In [ ]:
!pip install diffusers==0.31.0
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

device = "cuda"
#device = "mps"

Eksplorasi CLIP
---

In [ ]:
def setup_clip_model():
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    return model.to(device), processor

clip_model, clip_processor = setup_clip_model()
!nvidia-smi

### Image-Text Similarity

In [ ]:
def compute_image_text_similarity(model, processor, image, texts):
    # Prepare inputs
    inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Compute similarity scores
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds
    similarity_scores = (image_embeds @ text_embeds.T).squeeze()

    return similarity_scores.cpu().numpy()

# Test the function
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
texts = ["a photo of two cats", "a photo of a cat", "a photo of a dog", "a photo of a giraffe"]

similarities = compute_image_text_similarity(clip_model, clip_processor, image, texts)

for text, score in zip(texts, similarities):
    print(f"Similarity with '{text}': {score:.4f}")

Tugas
---
Cobalah menghitung image-text similarity pada gambar dan teks kalian sendiri

### Siapkan Pretrained

In [ ]:
sdv1_4 = "CompVis/stable-diffusion-v1-4"
sdv1_5 = "runwayml/stable-diffusion-v1-5"
sdv2_1b = "stabilityai/stable-diffusion-2-1-base"
sdv2_1 = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(sdv2_1b, torch_dtype=torch.float16)

In [ ]:
pipe

### Pindahkan Pipeline ke GPU

In [ ]:
pipe = pipe.to(device)
!nvidia-smi

### Hasilkan Gambar melalui Prompt

In [ ]:
prompt = "a high quality photograph of an astronaut rides a cow"
image = pipe(prompt).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
image

In [ ]:
generator = torch.Generator("cuda").manual_seed(33)

image = pipe(prompt, generator=generator).images[0]

image

Tugas
---
Cobalah pada prompt kalian sendiri

### Ubah Jumlah Langkah Difusi

In [ ]:
generator = torch.Generator("cuda").manual_seed(27)

image = pipe(prompt, num_inference_steps=30, generator=generator).images[0]

image

### Tugas

Cobalah langkah difusi kecil dan besar

### Integritas Gambar

Parameter `guidance_scale` adalah cara untuk meningkatkan kepatuhan terhadap teks serta kualitas sampel secara keseluruhan. Angka seperti `7` atau `8.5` memberikan hasil yang baik, jika Anda menggunakan angka yang sangat besar gambarnya mungkin terlihat bagus, namun kurang beragam.

In [ ]:
generator = torch.Generator("cuda").manual_seed(1)

image = pipe(prompt, generator=generator, guidance_scale = 3).images[0]

image

### Tugas

Cobalah integritas gambar kecil dan besar

### Lain-lain

Untuk menghasilkan banyak gambar untuk prompt yang sama, kita cukup menggunakan daftar dengan prompt yang sama yang diulang beberapa kali.

Pertama-tama mari kita tulis fungsi pembantu (`image_grid`) untuk menampilkan kisi-kisi gambar.

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

Sekarang, kita dapat membuat gambar grid setelah menjalankan pipeline dengan daftar 5 perintah.

In [ ]:
num_images = 5
prompt = ["sunny city landscape with style of makoto shinkai"] * num_images

images = pipe(prompt).images

grid = image_grid(images, rows=1, cols=num_images)
grid

Tugas
---
Cobalah menggunakan prompt dan jumlah gambar yang berbeda

Dan inilah cara membuat grid gambar sebanyak `n × m`.

In [ ]:
num_cols = 2
num_rows = 3

prompt = ["a proposed minimalist bedroom design"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

Tugas
---
Cobalah menggunakan prompt dan jumlah kalian sendiri

### Menghasilkan Gambar Non-Persegi

Secara default, algoritma ini menghasilkan gambar `512 × 512` piksel. Untuk mengganti default, gunakan argumen `height` dan `width`, sehingga dapat membuat gambar dalam rasio potret atau lanskap.

In [ ]:
prompt = "Balinese Sea with neoclassical painting style"

image = pipe(prompt, height=512, width=768).images[0]
image

Tugas
---
Cobalah menggunakan prompt dan ukuran kalian sendiri

# Tugas

Coba pikirkan sebuah prompt yang unik dan kreatif, untuk menghasilkan sebuah gambar 512x512.

Lalu bandingkan gambar yang dihasilkan untuk 3 pretrained model yang berbeda di bawah ini:
- stable-diffusion-v1-4
    - guidance_scale=3
    - guidance_scale=8
    - guidance_scale=15
- stable-diffusion-v1-5
    - guidance_scale=3
    - guidance_scale=8
    - guidance_scale=15
- stable-diffusion-2-1-base
    - guidance_scale=3
    - guidance_scale=8
    - guidance_scale=15

Lakukan analisis dan perbandingan berbagai gambar yang dihasilkan oleh model dan parameter yang berbeda.

Tugas
---
Cobalah cari dan kerjakan diffusion model jenis yang berbeda, misalnya image-to-image, atau image-to-video